In [ ]:
# Import the required libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD,Adam

In [ ]:
# Mount the google drive folder and load the Iris data set

from google.colab import drive

drive.mount('/content/gdrive', force_remount = True)
os.chdir('/content/gdrive/My Drive')

data = pd.read_csv("Iris.csv")

Mounted at /content/gdrive


In [ ]:
# Examine the dataframe for sanity

data.head(5)

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [ ]:
# 1. Separate the predictor and label columns
# 2. Encode the label column using one hot encoding
# 3. Perform train - test split

X = data[['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm']]
y = data['Species']

encoder =  LabelEncoder()
y = encoder.fit_transform(y)
y = pd.get_dummies(y).values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 123) 

In [ ]:
# Define and compose model

model_1 = Sequential()

# Define the layers
model_1.add(Dense(20, input_shape = (4, ), activation = 'tanh'))
model_1.add(Dense(10, activation = 'tanh'))
model_1.add(Dense(10, activation = 'tanh'))
model_1.add(Dense(3, activation = 'softmax'))

# Compile the model with the required optimizer, loss function and metric
model_1.compile(Adam(lr = 0.04), 'categorical_crossentropy', metrics = ['accuracy'])

# Print the model summary
model_1.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 20)                100       
_________________________________________________________________
dense_1 (Dense)              (None, 10)                210       
_________________________________________________________________
dense_2 (Dense)              (None, 10)                110       
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 33        
Total params: 453
Trainable params: 453
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Fit the model

history_1 = model_1.fit(X_train, y_train, epochs = 100, validation_data = (X_test, y_test))

Epoch 1/100
4/4 [==============================] - 1s 152ms/step - loss: 1.2280 - accuracy: 0.3556 - val_loss: 0.8389 - val_accuracy: 0.8333
Epoch 2/100
4/4 [==============================] - 0s 13ms/step - loss: 0.7936 - accuracy: 0.7848 - val_loss: 0.5158 - val_accuracy: 0.6333
Epoch 3/100
4/4 [==============================] - 0s 15ms/step - loss: 0.5088 - accuracy: 0.6810 - val_loss: 0.3306 - val_accuracy: 0.9667
Epoch 4/100
4/4 [==============================] - 0s 12ms/step - loss: 0.3783 - accuracy: 0.9204 - val_loss: 0.2236 - val_accuracy: 1.0000
Epoch 5/100
4/4 [==============================] - 0s 13ms/step - loss: 0.2543 - accuracy: 0.9538 - val_loss: 0.1595 - val_accuracy: 0.9667
Epoch 6/100
4/4 [==============================] - 0s 15ms/step - loss: 0.1509 - accuracy: 0.9417 - val_loss: 0.0813 - val_accuracy: 0.9667
Epoch 7/100
4/4 [==============================] - 0s 53ms/step - loss: 0.1703 - accuracy: 0.9352 - val_loss: 0.6269 - val_accuracy: 0.8000
Epoch 8/100
4/4 [==

In [ ]:
# Saving the simple model in  format

model_1.save("model_1.h5")

In [ ]:
# Convert the keras model to tensorflow.js

!pip install tensorflowjs 

     |████████████████████████████████| 71kB 3.6MB/s 
     |████████████████████████████████| 112kB 7.6MB/s 
  Found existing installation: tensorflow-hub 0.10.0
    Uninstalling tensorflow-hub-0.10.0:
      Successfully uninstalled tensorflow-hub-0.10.0


In [ ]:
# Convert the model to tfjs

!mkdir model_1

!tensorflowjs_converter --input_format keras model_1.h5 model/

2021-01-03 06:55:15.724983: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [ ]:
# Zip and download the model

!zip -r model.zip model 

from google.colab import files
files.download('model.zip')

  adding: model/ (stored 0%)
  adding: model/group1-shard1of1.bin (deflated 4%)
  adding: model/model.json (deflated 78%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Predict for a specific set of lengths

slf = 2
swf = 3
plf = 2
pwf = 5

testVal = np.array([slf, swf, plf, pwf]).reshape(1, 4)

classNames = ["Setosa", "Virginica", "Versicolor"]

print("Class is ", classNames[np.argmax(model_1.predict(testVal))])

Class is  Versicolor
